In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from google.colab import drive
drive.mount("/content/drive")

## This is my own dataset (i collect it from my whatsapp chat screenshot and from my personal gallery), so if you want to use it please reach me in Discord Amna#3650

In [ ]:
!unzip /content/drive/MyDrive/own_dataset/chat-non-chat.zip

In [ ]:
!mkdir data
!cp -r chat data/
!cp -r non data/

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'data', 
        target_size=(224, 224), 
        batch_size=32,
        class_mode='binary')


In [ ]:
def make_model(base_model):
    finetuning_model = tf.keras.Sequential([
        tf.keras.layers.InputLayer((224, 224, 3)),
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    return finetuning_model

base_layer = hub.KerasLayer("https://tfhub.dev/adityakane2001/regnety400mf_feature_extractor/1", trainable=False)

model = make_model(base_layer)


model.build(input_shape=(None, 224, 224, 3))
model.summary()

In [ ]:
model.compile(
    optimizer="adam",
    loss= "binary_crossentropy",
    metrics=["accuracy"]    
)

print("Training model")
model.fit(
    train_generator,
    epochs=5
)

In [ ]:
model.save("/content/drive/MyDrive/model_chat_non_chat/")

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  # x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a non chat")
  else:
    print(fn + " is a chat")
 